# PythonによるWebスクレイピング（動的Webページ）

以下では，サンプルページ

https://hirotakeyamazoe.jp/lecture/advexp/f1/

に対して，スクレイピングを行う例を示す．

このページでは，表示データを動的に生成しているため，ソースファイルには表示データが含まれない．そのため，実際に表示されるページのレンダリングを行ってから，その情報に対しスクレイピングを行う必要がある．

ここでは，Playwrightライブラリを利用してレンダリング後のHTMLページを取得し，Beautiful soupライブラリを用いてスクレイピングを行う例を示す．

### ライブラリのインストール

In [1]:
!pip install playwright beautifulsoup4 lxml_html_clean
!playwright install
!playwright install-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 MB 15.8 MB/s eta 0:00:00
173.9 MiB [] 0% 0.0s173.9 MiB [] 0% 45.4s173.9 MiB [] 0% 45.2s173.9 MiB [] 0% 31.9s173.9 MiB [] 0% 25.1s173.9 MiB [] 0% 18.8s173.9 MiB [] 0% 13.3s173.9 MiB [] 1% 9.5s173.9 MiB [] 1% 7.0s173.9 MiB [] 2% 5.1s173.9 MiB [] 3% 4.1s173.9 MiB [] 4% 3.7s173.9 MiB [] 5% 3.8s173.9 MiB [] 5% 3.5s173.9 MiB [] 6% 3.5s173.9 MiB [] 7% 3.2s173.9 MiB [] 8% 2.9s173.9 MiB [] 9% 2.8s173.9 MiB [] 10% 2.7s173.9 MiB [] 11% 2.6s173.9 MiB [] 12% 2.4s173.9 MiB [] 13% 2.3s173.9 MiB [] 14% 2.2s173.9 MiB [] 15% 2.1s173.9 MiB [] 16% 2.0s173.9 MiB [] 18% 2.0s173.9 MiB [] 19% 1.9s173.9 MiB [] 20% 1.8s173.9 MiB [] 21% 1.8s173.9 MiB [] 21% 1.9s173.9 MiB [] 22% 1.9s173.9 MiB [] 23% 1.8s173.9 MiB [] 25% 1.7s173.9 MiB [] 26% 1.6s173.9 MiB [] 27% 1.6s173.9 MiB [] 28% 1.6s173.9 MiB [] 29% 1.6s173.9 MiB [] 30% 1.5s173.9 MiB [] 31% 1.5s173.9 MiB [] 32% 1.5s173.9 MiB [] 33% 1.5s173.9 MiB [] 34% 1.5s173.9 MiB [] 35% 1.4s173.9 MiB [] 36% 1.4s173.9 MiB

### レンダリング結果の取得と表示

In [2]:
from playwright.async_api import async_playwright
from IPython.core.display import display, HTML

url = "https://hirotakeyamazoe.jp/lecture/advexp/f1/"


# Playwright を用いてページを取得してレンダリング
async def get_page_html(url):
    async with async_playwright() as p:
        # Chromiumブラウザを起動
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        # ページにアクセスし，ネットワークが落ち着くまで待つ
        await page.goto(url, wait_until="networkidle")

        # レンダリング後のHTMLを取得
        html = await page.content()

        # ブラウザを閉じる
        await browser.close()

        return html

# レンダリング結果の取得
html = await get_page_html(url)

# レンダリングされたHTMLを可視化
display(HTML(html))


Position,Name,Code,Nationality,Car,Points
1,Max Verstappen,VER,NED,Red Bull Racing Honda RBPT,393
2,Lando Norris,NOR,GBR,McLaren Mercedes,331
3,Charles Leclerc,LEC,MON,Ferrari,307
4,Oscar Piastri,PIA,AUS,McLaren Mercedes,262
5,Carlos Sainz,SAI,ESP,Ferrari,244
6,George Russell,RUS,GBR,Mercedes,192
7,Lewis Hamilton,HAM,GBR,Mercedes,190
8,Sergio Perez,PER,MEX,Red Bull Racing Honda RBPT,151
9,Fernando Alonso,ALO,ESP,Aston Martin Aramco Mercedes,62
10,Nico Hulkenberg,HUL,GER,Haas Ferrari,31


### レンダリング結果の表示とファイル保存

保存したファイルを用いると，ブラウザの"デベロッパーツール"などを用いて，抽出したい場所を探索することができるため，これ以降は，「PythonによるWebスクレイピング」と同様の手順となる．

In [3]:
print(html)

# レンダリング結果をHTMLファイルとして保存
with open("rendered_page.html", "w", encoding="utf-8") as f:
    f.write(html)

<!DOCTYPE html><html lang="ja"><head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Driver Standings</title>
    <link rel="stylesheet" href="https://hirotakeyamazoe.jp/lecture/advexp/f1/styles.css">
</head>
<body>
    <div class="container">
        <h1>2024 Driver Standings</h1>
        <table id="driver-standings">
            <thead>
                <tr>
                    <th>Position</th>
                    <th>Name</th>
                    <th>Code</th>
                    <th>Nationality</th>
                    <th>Car</th>
                    <th>Points</th>
                </tr>
            </thead>
            <tbody><tr><td>1</td><td>Max&nbsp;Verstappen</td><td>VER</td><td>NED</td><td>Red Bull Racing Honda RBPT</td><td>393</td></tr><tr><td>2</td><td>Lando&nbsp;Norris</td><td>NOR</td><td>GBR</td><td>McLaren Mercedes</td><td>331</td></tr><tr><td>3</td><td>Charles&nbsp;Leclerc</td><td>LEC</td><td>MON</td><td

### Beautiful soupによる解析（表のデータの抽出と表示）
ここでは，tableの情報を取得し，各行，列のデータを抽出して，DataFrameとして保存している．

In [4]:
from bs4 import BeautifulSoup
import pandas as pd

pd.set_option('display.width', 100)       # 出力横幅を100に設定


# レンダリング結果をBeautifulSoupで解析
soup = BeautifulSoup(html, 'html.parser')

standings_table = soup.find(id ='driver-standings')  # Adjust the class if necessary

# tableのヘッダ部の抽出
headers = [th.text.strip() for th in standings_table.find_all('th')]

# 行ごとのデータを配列として取得し，ヘッダ行をスキップ
rows = standings_table.find_all('tr')[1:]
data = [] # 保存用リストを初期化
for row in rows: # 各行に関する処理
    cols = [td.text.strip() for td in row.find_all('td')] # 各列の抽出
    data.append(cols) # 保存用リストに追加

# リストからDataFrameへの変換
df = pd.DataFrame(data, columns=headers)

# DataFrameの表示
print(df)

   Position              Name Code Nationality                           Car Points
0         1    Max Verstappen  VER         NED    Red Bull Racing Honda RBPT    393
1         2      Lando Norris  NOR         GBR              McLaren Mercedes    331
2         3   Charles Leclerc  LEC         MON                       Ferrari    307
3         4     Oscar Piastri  PIA         AUS              McLaren Mercedes    262
4         5      Carlos Sainz  SAI         ESP                       Ferrari    244
5         6    George Russell  RUS         GBR                      Mercedes    192
6         7    Lewis Hamilton  HAM         GBR                      Mercedes    190
7         8      Sergio Perez  PER         MEX    Red Bull Racing Honda RBPT    151
8         9   Fernando Alonso  ALO         ESP  Aston Martin Aramco Mercedes     62
9        10   Nico Hulkenberg  HUL         GER                  Haas Ferrari     31
10       11      Yuki Tsunoda  TSU         JPN                 RB Honda RBPT